In [1]:
from spimy2 import BSBIndexConstuction
import pandas as pd

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from spimi import SPIMI

[nltk_data] Downloading package punkt to /home/ambar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# testeo
path = './dataset/'
data_path = path + 'spotify_songs_test2.csv'

data = pd.read_csv(data_path)

In [5]:
spimi = SPIMI(size_per_block=10240*4,
              output_folder='./blocks/',
              ram_limit=1024*1024*1024*4,
              size_per_block_out= 1024*4)

spimi.BSBIndexConstuction(data)

['./blocks/block_0.txt',
 './blocks/block_1.txt',
 './blocks/block_2.txt',
 './blocks/block_3.txt',
 './blocks/block_4.txt',
 './blocks/block_5.txt',
 './blocks/block_6.txt',
 './blocks/block_7.txt',
 './blocks/block_8.txt',
 './blocks/block_9.txt',
 './blocks/block_10.txt',
 './blocks/block_11.txt',
 './blocks/block_12.txt',
 './blocks/block_13.txt',
 './blocks/block_14.txt',
 './blocks/block_15.txt',
 './blocks/block_16.txt',
 './blocks/block_17.txt',
 './blocks/block_18.txt',
 './blocks/block_19.txt',
 './blocks/block_20.txt',
 './blocks/block_21.txt',
 './blocks/block_22.txt',
 './blocks/block_23.txt',
 './blocks/block_24.txt',
 './blocks/block_25.txt',
 './blocks/block_26.txt',
 './blocks/block_27.txt',
 './blocks/block_28.txt',
 './blocks/block_29.txt',
 './blocks/block_30.txt',
 './blocks/block_31.txt',
 './blocks/block_32.txt',
 './blocks/block_33.txt',
 './blocks/block_34.txt',
 './blocks/block_35.txt',
 './blocks/block_36.txt',
 './blocks/block_37.txt',
 './blocks/block_38.tx

In [7]:
import numpy as np
import os
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import regex as re
import bisect

In [ ]:
class CosineSimilaritySearch:
    def __init__(self, block_folder):
        self.block_folder = block_folder
        self.stopwords = stopwords.words('spanish')  # Ajusta el idioma si es necesario
        self.stemmer = SnowballStemmer(language='spanish')  # Ajusta el idioma si es necesario

    def parse_docs(self, doc):
        """Procesa el texto en el documento usando un enfoque similar al usado en SPIMI."""
        words = []
        col_text = [
            'lyrics', 'track_name', 'track_artist', 
            'track_album_name', 'playlist_name', 
            'playlist_genre', 'playlist_subgenre'
        ]
        
        # Asegurarse de que los valores son cadenas y unirlos
        texto = ' '.join(str(doc[col]) if isinstance(doc[col], str) else '' for col in col_text).lower()
        texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)

        language_map = {
            'es': 'spanish',
            'en': 'english',
            'fr': 'french',
            'de': 'german',
            'it': 'italian'
            # Agrega más idiomas según sea necesario
        }
        
        ln = language_map.get(doc.get('language', 'en'), 'english')
        
        for word in texto.split():
            if word not in stopwords.words(ln):
                words.append(word)

        stemmer = SnowballStemmer(language=ln)
        return [stemmer.stem(word) for word in words]
    
    def calculate_query_vector(self, query_terms, df_dict, num_docs):
        """Calcula el vector TF-IDF para la consulta."""
        tf_query = defaultdict(int)
        for term in query_terms:
            tf_query[term] += 1
        
        query_vector = {}
        for term, tf in tf_query.items():
            idf = np.log((num_docs / df_dict[term])) if term in df_dict else 0
            query_vector[term] = tf * idf
        return query_vector

    def load_block_terms(self, query_terms):
        """Carga solo los términos relevantes de los bloques para la consulta usando búsqueda binaria."""
        term_postings = {}
        df_dict = {}
        num_docs = 0

        for filename in os.listdir(self.block_folder):
            with open(os.path.join(self.block_folder, filename), 'r') as file:
                for line in file:
                    term, rest = line.split(" (DF: ")
                    if self.binary_search(query_terms, term):
                        df, postings = rest.split("): ")
                        df = int(df)
                        df_dict[term] = df
                        num_docs += df
                        
                        term_postings[term] = []
                        postings_list = postings.strip().split("), (")
                        for posting in postings_list:
                            doc_id, tf = map(int, posting.strip("()").split(", "))
                            term_postings[term].append((doc_id, tf))
        return term_postings, df_dict, num_docs

    def binary_search(self, sorted_list, item):
        """Realiza búsqueda binaria para comprobar si un elemento existe en una lista ordenada."""
        index = bisect.bisect_left(sorted_list, item)
        return index < len(sorted_list) and sorted_list[index] == item

    def cosine_similarity(self, query_vector, doc_vectors):
        """Calcula la similitud de coseno entre la consulta y cada documento, con depuración."""
        doc_scores = defaultdict(float)
        query_norm = math.sqrt(sum(weight**2 for weight in query_vector.values()))
        
        # Acumula pesos para cada documento y muestra detalles para depuración
        print("Vector de la consulta:", query_vector)
        for term, query_weight in query_vector.items():
            if term in doc_vectors:
                for doc_id, doc_tf in doc_vectors[term]:
                    doc_weight = doc_tf * query_weight
                    doc_scores[doc_id] += doc_weight
        
        # Imprimir los puntajes antes de la normalización para ver el valor de doc_id=64
        print("\nPuntajes antes de la normalización:")
        for doc_id, score in doc_scores.items():
            print(f"Documento {doc_id}: {score}")
        
        # Calcular la norma del vector del documento y normalizar
        for doc_id in doc_scores:
            doc_norm = math.sqrt(sum(
                (doc_tf * query_vector.get(term, 0))**2 
                for term in query_vector
                for doc_id_in_posting, doc_tf in doc_vectors.get(term, [])
                if doc_id_in_posting == doc_id
            ))
            if doc_norm != 0:
                doc_scores[doc_id] /= (query_norm * doc_norm)
            else:
                doc_scores[doc_id] = 0

        # Imprimir los puntajes después de la normalización
        print("\nPuntajes después de la normalización:")
        for doc_id, score in doc_scores.items():
            print(f"Documento {doc_id}: {score}")
        
        return sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)


    def get_top_k_similar_documents(self, query, data, k=5):
        # Procesar la consulta y realizar la búsqueda
        query_terms = self.preprocess(query)
        term_postings, df_dict, num_docs = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict, num_docs)
        
        # Calcular las similitudes y ordenar los documentos
        doc_scores = self.cosine_similarity(query_vector, term_postings)
        doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)  # Asegurar el orden descendente de similitud
        print(doc_scores)

        # Obtener los primeros k documentos más similares
        top_k_docs = doc_scores[:k][:k]
        
        
        # Crear una lista para almacenar los resultados con detalles del documento
        results = []
        for doc_id, score in top_k_docs:
            try:
                # Acceder a los detalles del documento con `iloc`
                doc_details = data.iloc[doc_id].to_dict()
                doc_details["Cosine Similarity Score"] = score
                results.append(doc_details)
            except IndexError:
                print(f"El documento con ID {doc_id} no está disponible en los datos.")

        # Convertir los resultados en un DataFrame para mejor visualización y seleccionar solo columnas relevantes
        results_df = pd.DataFrame(results)
        results_df = results_df[["track_name", "track_artist", "lyrics", "Cosine Similarity Score"]]  # Columnas esenciales
        results_df.columns = ["Song Title", "Artist", "Lyrics", "Similarity Score"]  # Renombrar columnas
        results_df.index = range(1, len(results) + 1)  # Índice 1, 2, ..., k para la tabla
        return results_df



    def search(self, query):
        query_terms = self.preprocess(query)
        term_postings, df_dict, num_docs = self.load_block_terms(query_terms)
        query_vector = self.calculate_query_vector(query_terms, df_dict, num_docs)
        return self.cosine_similarity(query_vector, term_postings)


In [26]:
import pandas as pd


In [75]:
print(data.iloc[64])

track_id                                               02138lFv3Bzncr6ScNbLAF
track_name                                                   Rattle Your Cage
track_artist                                                   Skrizzly Adams
lyrics                      Well I know you've been talking somebody When ...
track_popularity                                                           44
track_album_id                                         3RcttHMKlZ7K1ovxIANPd0
track_album_name                                                    Young Man
track_album_release_date                                           2019-11-15
playlist_name                                       Supernatural Classic Rock
playlist_id                                            6oghIlByD49KFGNmNU8GSH
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

In [94]:


block_folder = './blocks/'
search_engine = CosineSimilaritySearch(block_folder)
results_df = search_engine.get_top_k_similar_documents("yea you just can't walk away", data, k=5)
print("Top K documentos más similares:")
# print(results_df)

Vector de la consulta: {'yea': 3.767822656454056, 'you': 3.4111477125153233, 'just': 4.61512051684126, 'can': 4.61512051684126, 't': 4.327438444389479, 'walk': 1.0693419063679965, 'away': 0.5662383286959161}

Puntajes antes de la normalización:
Documento 64: 58.159361473220606
Documento 247: 21.487342064024205
Documento 479: 11.303467969362167
Documento 635: 10.183874094662036
Documento 737: 3.767822656454056
Documento 892: 11.303467969362167
Documento 14: 3.4111477125153233
Documento 143: 6.822295425030647
Documento 227: 13.644590850061293
Documento 256: 6.822295425030647
Documento 484: 6.822295425030647
Documento 628: 3.4111477125153233
Documento 724: 6.822295425030647
Documento 910: 3.4111477125153233
Documento 996: 11.437415941871906
Documento 109: 169.05778140364146
Documento 858: 4.61512051684126
Documento 875: 4.61512051684126
Documento 63: 4.61512051684126
Documento 740: 4.61512051684126
Documento 439: 17.309753777557916
Documento 480: 8.654876888778958
Documento 836: 8.6548768